In [24]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_timestamp, col, count, countDistinct, sum, avg, lit, to_timestamp, coalesce
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DoubleType
from delta.tables import *
from delta import *
from delta import configure_spark_with_delta_pip


In [3]:
builder = SparkSession \
        .builder \
        .appName('healthcare_transformation')\
        .master('local')\
        .config("spark.driver.memory","2g")\
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")\
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [4]:
spark

In [ ]:
bronze_folder = "bronze"

In [5]:
patients = spark.read \
                .format("delta").load(f"{bronze_folder}\\healthcare_delta\\patients")

In [6]:
patients.printSchema()

root
 |-- patients_id: string (nullable = true)
 |-- BIRTHDATE: date (nullable = true)
 |-- DEATHDATE: date (nullable = true)
 |-- PREFIX: string (nullable = true)
 |-- FIRST: string (nullable = true)
 |-- LAST: string (nullable = true)
 |-- SUFFIX: string (nullable = true)
 |-- MAIDEN: string (nullable = true)
 |-- MARITAL: string (nullable = true)
 |-- RACE: string (nullable = true)
 |-- ETHNICITY: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- BIRTHPLACE: string (nullable = true)
 |-- ADDRESS: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- COUNTY: string (nullable = true)
 |-- ZIP: integer (nullable = true)
 |-- LAT: double (nullable = true)
 |-- LON: double (nullable = true)



In [7]:
encounters = spark.read \
                .format("delta").load(f"{bronze_folder}\\healthcare_delta\\encounters") \
                .withColumnRenamed("REASONCODE", "REASONCODE_EN") \
                .withColumnRenamed("REASONDESCRIPTION", "REASONDESCRIPTION_EN")

In [8]:
encounters.printSchema()

root
 |-- encounters_id: string (nullable = true)
 |-- START: timestamp (nullable = true)
 |-- STOP: timestamp (nullable = true)
 |-- PATIENT_ID: string (nullable = true)
 |-- ORGANIZATION: string (nullable = true)
 |-- PAYER: string (nullable = true)
 |-- ENCOUNTERCLASS: string (nullable = true)
 |-- CODE: integer (nullable = true)
 |-- DESCRIPTION: string (nullable = true)
 |-- BASE_ENCOUNTER_COST: double (nullable = true)
 |-- TOTAL_CLAIM_COST: double (nullable = true)
 |-- PAYER_COVERAGE: double (nullable = true)
 |-- REASONCODE_EN: long (nullable = true)
 |-- REASONDESCRIPTION_EN: string (nullable = true)



In [9]:
procedure = spark.read \
                .format("delta").load(f"{bronze_folder}\\healthcare_delta\\procedure")

In [10]:
procedure.printSchema()

root
 |-- START: timestamp (nullable = true)
 |-- STOP: timestamp (nullable = true)
 |-- PATIENT_ID: string (nullable = true)
 |-- ENCOUNTER_ID: string (nullable = true)
 |-- CODE: long (nullable = true)
 |-- DESCRIPTION: string (nullable = true)
 |-- BASE_COST: integer (nullable = true)
 |-- REASONCODE: long (nullable = true)
 |-- REASONDESCRIPTION: string (nullable = true)



In [11]:
payers = spark.read \
            .format("delta").load(f"{bronze_folder}\\healthcare_delta\\payers") \
            .withColumnRenamed("CITY", "PAYER_CITY")

In [12]:
payers.printSchema()

root
 |-- PAYER_ID: string (nullable = true)
 |-- PAYER_NAME: string (nullable = true)
 |-- ADDRESS: string (nullable = true)
 |-- PAYER_CITY: string (nullable = true)
 |-- STATE_HEADQUARTERED: string (nullable = true)
 |-- ZIP: integer (nullable = true)
 |-- PHONE: string (nullable = true)



In [13]:
combined_pe = patients.join(encounters, patients.patients_id == encounters.PATIENT_ID) \
                      .join(procedure, encounters.PATIENT_ID == procedure.PATIENT_ID) \
                      .join(payers, encounters.PAYER == payers.PAYER_ID)
                      

In [14]:
combined_pe.printSchema()

root
 |-- patients_id: string (nullable = true)
 |-- BIRTHDATE: date (nullable = true)
 |-- DEATHDATE: date (nullable = true)
 |-- PREFIX: string (nullable = true)
 |-- FIRST: string (nullable = true)
 |-- LAST: string (nullable = true)
 |-- SUFFIX: string (nullable = true)
 |-- MAIDEN: string (nullable = true)
 |-- MARITAL: string (nullable = true)
 |-- RACE: string (nullable = true)
 |-- ETHNICITY: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- BIRTHPLACE: string (nullable = true)
 |-- ADDRESS: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- COUNTY: string (nullable = true)
 |-- ZIP: integer (nullable = true)
 |-- LAT: double (nullable = true)
 |-- LON: double (nullable = true)
 |-- encounters_id: string (nullable = true)
 |-- START: timestamp (nullable = true)
 |-- STOP: timestamp (nullable = true)
 |-- PATIENT_ID: string (nullable = true)
 |-- ORGANIZATION: string (nullable = true)
 |-- PAYER: string (nulla

In [25]:
final_df = combined_pe.select("patients_id", "encounters_id", "FIRST", "LAST","GENDER", "CITY","BIRTHDATE","DEATHDATE","ENCOUNTERCLASS","REASONCODE", "REASONDESCRIPTION",
                             "BASE_COST", "BASE_ENCOUNTER_COST", "TOTAL_CLAIM_COST", "PAYER", "PAYER_COVERAGE","PAYER_NAME" 
                   )\
                   .withColumn("INGESTION_DATE", current_timestamp())\
                   .withColumn("MODIFICATION_DATE", current_timestamp())\
                   .withColumn("SOURCE", lit("Kaggle"))

In [26]:
final_df.printSchema()

root
 |-- patients_id: string (nullable = true)
 |-- encounters_id: string (nullable = true)
 |-- FIRST: string (nullable = true)
 |-- LAST: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- BIRTHDATE: date (nullable = true)
 |-- DEATHDATE: date (nullable = true)
 |-- ENCOUNTERCLASS: string (nullable = true)
 |-- REASONCODE: long (nullable = true)
 |-- REASONDESCRIPTION: string (nullable = true)
 |-- BASE_COST: integer (nullable = true)
 |-- BASE_ENCOUNTER_COST: double (nullable = true)
 |-- TOTAL_CLAIM_COST: double (nullable = true)
 |-- PAYER: string (nullable = true)
 |-- PAYER_COVERAGE: double (nullable = true)
 |-- PAYER_NAME: string (nullable = true)
 |-- INGESTION_DATE: timestamp (nullable = false)
 |-- MODIFICATION_DATE: timestamp (nullable = false)
 |-- SOURCE: string (nullable = false)



In [27]:
final_df_dropped = final_df.dropDuplicates(['encounters_id'])

In [19]:
final_df_dropped.count()

26294

In [29]:
final_df_dropped.show()

+--------------------+--------------------+-----------+-------------+------+----------+----------+----------+--------------+----------+--------------------+---------+-------------------+----------------+--------------------+--------------+--------------------+--------------------+--------------------+------+
|         patients_id|       encounters_id|      FIRST|         LAST|GENDER|      CITY| BIRTHDATE| DEATHDATE|ENCOUNTERCLASS|REASONCODE|   REASONDESCRIPTION|BASE_COST|BASE_ENCOUNTER_COST|TOTAL_CLAIM_COST|               PAYER|PAYER_COVERAGE|          PAYER_NAME|      INGESTION_DATE|   MODIFICATION_DATE|SOURCE|
+--------------------+--------------------+-----------+-------------+------+----------+----------+----------+--------------+----------+--------------------+---------+-------------------+----------------+--------------------+--------------+--------------------+--------------------+--------------------+------+
|5e129b38-3d63-369...|0002c38a-54e9-078...|  Jolene291|     Kling921| 

In [30]:
try:
    final_df_dropped.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save("silver\\healthcare_trans")
    print("table created")
except Exception as e:
    print("Table creation failed")
    print (e)

table created


In [31]:
spark.stop

<bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x000001F33A2F06D0>>